# S&P 500 Portfolio Builder

This project helps you decide **how much money to invest** in each company in the **S&P 500** (a list of the 500 biggest companies in the U.S.). It also tells you **how many shares** of each company to buy in order to have equal share of each company.


---

## **What Does It Do?**
1. **Loads Company Data**: It starts by loading a list of S&P 500 companies from a file.
2. **Gets Stock Prices**: It checks the latest stock prices and company sizes (called "market cap") using Yahoo Finance.
3. **Calculates Investments**:
   - Bigger companies get more money.
   - Smaller companies get less money.
4. **Saves Results**: It creates an Excel file showing:
   - How much to invest in each company.
   - How many shares to buy.


--------------------------------------------------------------------------------------------------------------------------------------------

## **Why Is This Useful?**
- **Diversification**: It spreads your money across many companies, so you don’t put all your eggs in one basket, as you might lose some.
- **Automation**: It does all the math for you, saving time.
- **Transparency**: You can see exactly where your money is going.


--------------------------------------------------------------------------------------------------------------------------------------------

## **How to Use**:
1. Run the notebook (click "Run All").
2. Check the Excel file it creates for your investment plan.
3. If any companies are skipped (due to errors), you can review them in a separate file.



In [1]:
import numpy as np, pandas as pd, yfinance as yf, math, time
from datetime import datetime
from requests.exceptions import RequestException

In [14]:
def format_market_cap(value):
    if pd.isna(value) or value == None:
        return "N/A"
        
    try: 
        value = float(value)
    except(ValueError, TypeError):
        return "N/A"

    if value >= 1e12:
        return f"{value/1e12:.2f} Trillion"
    elif value >= 1e9:
        return f"{value/1e9:.2f} Billion"
    elif value >= 1e6:
        return f"{value/1e6:.2f} Million"
    else: return f"{value:.2f}"

In [6]:
def fetch_ticker_data(tickers):
    for attempt in range(MAX_RETRIES):
        try:
            data = yf.download(
                tickers,
                period="1d",
                interval="1d",
                group_by='ticker',
                progress=False,
                threads=True
            )
            return data
        except RequestException as e:
            if attempt < MAX_RETRIES - 1:
                print(f"Retrying batch ({attempt+1}/{MAX_RETRIES})...")
                time.sleep(RETRY_DELAY * (attempt + 1))
                continue
            raise
    return pd.DataFrame()

In [7]:
PORTFOLIO_VALUE = 1000000
BATCH_SIZE = 30  # Reduced batch size for rate limiting
DELAY = 5  # Increased delay between batches
MAX_RETRIES = 3  # Retry failed requests
RETRY_DELAY = 10  # Seconds between retries

In [8]:
try:
    df = pd.read_csv('sp500_companies.csv')
    raw_tickers = df['Symbol'].dropna().unique().tolist()
    tickers = [t.replace('.','-') for t in raw_tickers]
    print(f"Loaded {len(tickers)} from CSV")
except Exception as e:
    print(f"Error reading CSV file: {e}")
    tickers = []

valid_stocks = []
skipped_tickers = []
total_batches = (len(tickers) - 1) // BATCH_SIZE + 1

for batch_num in range(total_batches):
    start_idx = batch_num * BATCH_SIZE
    end_idx = start_idx + BATCH_SIZE
    batch = tickers[start_idx : end_idx]
    
    print(f"\nProcessing batch {batch_num+1}/{total_batches} ({len(batch)} tickers)")

    try:
        price_data = fetch_ticker_data(batch)

        for symbol in batch:
            try:
                if symbol not in price_data.columns.get_level_values(0):
                    raise ValueError(f"No price data columns for {symbol}")
                
                symbol_data = price_data[symbol]
                if len(symbol_data) < 1:
                    raise ValueError(f"Empty price data for {symbol}")
                
                close_price = symbol_data['Close'].iloc[-1]
                if pd.isna(close_price):
                    raise ValueError(f"Invalid price for {symbol}")

                ticker = yf.Ticker(symbol)
                info = ticker.info
                market_cap = info.get('marketCap', None)
                
                valid_stocks.append({
                    'Symbol': symbol,
                    'Price': close_price,
                    'Market Cap': market_cap,
                    'Formatted Market Cap': format_market_cap(market_cap)
                })
                
            except Exception as e:
                print(f"Skipped {symbol}: {str(e)}")
                skipped_tickers.append((symbol, str(e)))
                
        time.sleep(DELAY)
        
    except Exception as batch_error:
        print(f"Batch {batch_num+1} failed: {str(batch_error)}")
        skipped_tickers.extend([(t, "Batch failed") for t in batch])
        continue

if not valid_stocks:
    raise ValueError("No valid stock data retrieved. Check tickers and connection.")

Loaded 502 from CSV

Processing batch 1/17 (30 tickers)
YF.download() has changed argument auto_adjust default to True

Processing batch 2/17 (30 tickers)

Processing batch 3/17 (30 tickers)

Processing batch 4/17 (30 tickers)

Processing batch 5/17 (30 tickers)

Processing batch 6/17 (30 tickers)

Processing batch 7/17 (30 tickers)

Processing batch 8/17 (30 tickers)

Processing batch 9/17 (30 tickers)

Processing batch 10/17 (30 tickers)

Processing batch 11/17 (30 tickers)

Processing batch 12/17 (30 tickers)



1 Failed download:
['WDC']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')


Skipped WDC: Invalid price for WDC

Processing batch 13/17 (30 tickers)

Processing batch 14/17 (30 tickers)

Processing batch 15/17 (30 tickers)

Processing batch 16/17 (30 tickers)

Processing batch 17/17 (22 tickers)


In [12]:
# Process results
if not valid_stocks:
    raise ValueError("No valid stock data retrieved. Check tickers and connection.")


holdings = pd.DataFrame(valid_stocks)


# Custom function to convert formatted market cap to numerical value
def convert_market_cap(value):
    if "Trillion" in value:
        return float(value.replace(" Trillion", "")) * 1e12
    elif "Billion" in value:
        return float(value.replace(" Billion", "")) * 1e9
    elif "Million" in value:
        return float(value.replace(" Million", "")) * 1e6
    else:
        return float(value) 

# Apply the function to the "Formatted Market Cap" column
holdings["Market Cap"] = holdings["Formatted Market Cap"].apply(convert_market_cap)

total_market_cap = holdings["Market Cap"].sum()
PORTFOLIO_VALUE = 1_000_000  

holdings["Weight"] = holdings["Market Cap"] / total_market_cap
holdings["Amount to Invest"] = holdings["Weight"] * PORTFOLIO_VALUE
holdings["Shares to Buy"] = (holdings["Amount to Invest"] / holdings["Price"]).apply(math.floor)

In [13]:
if skipped_tickers:
    skipped_df = pd.DataFrame(skipped_tickers, columns=['Ticker', 'Reason'])
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    skipped_filename = f"skipped_tickers_{timestamp}.csv"
    skipped_df.to_csv(skipped_filename, index=False)
    print(f"\nSaved {len(skipped_df)} skipped tickers to {skipped_filename}")

output_filename = f"portfolio_allocation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
    holdings.to_excel(writer, index=False, sheet_name='Allocation')
    
    workbook = writer.book
    worksheet = writer.sheets['Allocation']
    
    header_format = workbook.add_format({
        'bold': True, 'fg_color': '#4F81BD', 'font_color': 'white', 
        'border': 1, 'text_wrap': True
    })
    
    for col_num, value in enumerate(holdings.columns.values):
        worksheet.write(0, col_num, value, header_format)
    
    worksheet.set_column('A:A', 16)
    worksheet.set_column('B:B', 16)
    worksheet.set_column('C:C', 16)
    worksheet.set_column('D:D', 16)
    worksheet.set_column('E:E', 16)
    worksheet.set_column('F:F', 16)

print(f"\nSuccessfully processed {len(holdings)}/{len(tickers)} stocks")
print(f"Final results saved to {output_filename}")


Saved 1 skipped tickers to skipped_tickers_20250319_234950.csv

Successfully processed 501/502 stocks
Final results saved to portfolio_allocation_20250319_234950.xlsx
